# Imports

In [58]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

# Read Data

In [5]:
test_data = pd.read_csv(filepath_or_buffer='data/test.csv', nrows=100)
train_data = pd.read_csv(filepath_or_buffer='data/train.csv', nrows=100)

# Data Preprocessing/Transformations

## Explore

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    100 non-null    int64  
 1   Age                   99 non-null     float64
 2   Gender                100 non-null    object 
 3   Annual Income         94 non-null     float64
 4   Marital Status        99 non-null     object 
 5   Number of Dependents  92 non-null     float64
 6   Education Level       100 non-null    object 
 7   Occupation            65 non-null     object 
 8   Health Score          94 non-null     float64
 9   Location              100 non-null    object 
 10  Policy Type           100 non-null    object 
 11  Previous Claims       71 non-null     float64
 12  Vehicle Age           100 non-null    float64
 13  Credit Score          86 non-null     float64
 14  Insurance Duration    100 non-null    float64
 15  Policy Start Date     10

In [13]:
def cleanse_column_names(data):
    new_cols = []
    for col in data.columns:
        new_cols.append(col.lower().replace(' ', '_'))
    data.columns = new_cols
    return data.columns

In [41]:
cleanse_column_names(test_data)

Index(['id', 'age', 'gender', 'annual_income', 'marital_status',
       'number_of_dependents', 'education_level', 'occupation', 'health_score',
       'location', 'policy_type', 'previous_claims', 'vehicle_age',
       'credit_score', 'insurance_duration', 'policy_start_date',
       'customer_feedback', 'smoking_status', 'exercise_frequency',
       'property_type'],
      dtype='object')

In [78]:
def create_pipeline(numeric_features=None, ordinal_features=None, ordinal_categories=None, category_features=None):
    
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())   
    ])

    ordinal_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('encoder', OrdinalEncoder(categories=[ordinal_categories['exercise_frequency']], handle_unknown='use_encoded_value', unknown_value=-1))
    ])

    category_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
    ])

    # date_transformer = Pipeline(steps=[
        
    # ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric_features', numeric_transformer, numeric_features),
            ('ordinal_features', ordinal_transformer, ordinal_features),
            ('category_features', category_transformer, category_features)
        ])

    full_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', LGBMRegressor())
    ])
    
    return full_pipeline

In [60]:
train_data.drop('id', axis=1, inplace=True)
# test_data.drop('id', axis=1, inplace=True)

target = 'premium_amount'
X = train_data.drop(target, axis=1)
y = train_data[target]

y_log = np.log1p(y)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.20, random_state=42)

In [79]:
numeric_features = X.select_dtypes(include='number').columns

category_features = [
    'marital_status',
    'occupation',
    'location',
    'property_type',
    'education_level',
    'policy_type',
    'customer_feedback',
    'gender',
    'smoking_status'
]

ordinal_features = ['exercise_frequency']

ordinal_categories = {
    'exercise_frequency': ['Rarely', 'Monthly', 'Weekly', 'Daily']
}

# date_features = ['policy_start_date']

## Predict

In [80]:
pipeline = create_pipeline(
    numeric_features=numeric_features, 
    ordinal_features=ordinal_features,
    ordinal_categories=ordinal_categories,
    category_features=category_features
    )

pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

/home/vscode/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012919 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 176
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 24
[LightGBM] [Info] Start training from score 6.728474
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

/home/vscode/.local/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/vscode/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/vscode/.local/lib/python3.12/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [81]:
y_pred

array([7.0748177 , 7.20358495, 6.57926533, 6.40207167, 7.04607891,
       7.4887313 , 7.5311008 , 7.16906169, 7.55127389, 7.26680445,
       6.83460989, 6.50593625, 7.16623544, 7.63173015, 7.18470309,
       8.00625756, 7.76507682, 6.6117426 , 7.27346532, 6.19378226])